<a href="https://www.ired.info/"> <img src="file/ired_logo.png" alt="ired" style="width: 150px;"/> </a>

#  Object Classification- Train Model  物件分類 - 訓練模型

You will train your image classifier to detect two classes ``stop`` and ``press``, 
which you will use for pressing hand sanitizer collisions. 
<br> A popular deep learning library *PyTorch* is used in training the classification model.

### Put dataset to PyTorch 將數據載入至PyTorch

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

You will attach transforms from the pytorch package to prepare the data for training.  

In [ ]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

### Split dataset into **train set** and **test set** 將數據分為**訓練集**和**測試集**

You split the dataset into *training set* and *test set*.  The test set will be used to verify the accuracy of the model you train.
<br> 10 images will be 

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

### Create data loaders to load data in batches 以批次讀取數據

You will create two ``DataLoader``, which is used for shuffling data, and loading the samples in parallel for faster process.

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=4,
)

### Define the Neural Network 定義神經網絡

Now, we define the neural network we'll be training. PyTorch provides a collection of pre-trained models.

You will use **transfer learning** (遷移學習) on the pre-trained model (預訓練模型) **resnet18**, trained on millions of images alreadu
<br>Important features that were learned in the original training of the pre-trained model are re-usable for the new task.  

In [ ]:
model = models.resnet18(pretrained=True)

The ``resnet18`` model was originally trained for a dataset that had **1000 class labels**, but your dataset only has ``stop`` and ``press`` labels. 
<br> You will replace the final layer with a new, untrained layer that has only two outputs.  

In [ ]:
model.fc = torch.nn.Linear(512, 2)

Finally, your transfer the model to execute it on the GPU

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Train the neural network 訓練神經網絡

You will train the neural network for **30 epochs (迭代)** , only the best performing model will be saved.

An epoch means every image in the data has been processed once.
<br> Training with 50 images of each class should be finished within **5 minutes**. 
<br> More images will take longer time to comlpete the training. 

Wait for the training to initiate, The figure below shows the accuracy of each epoch.

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model_resnet18.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch+1, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

Now, you have successfully train a ``resnet18`` model. Move to the next notebook to see your training result.
<br> You shuold see your model ``best_model_resnet18.pth``.

--End--